# What is Docker?

Lets download and find out ;)

## 1
[Download Docker](https://download.docker.com/win/stable/InstallDocker.msi)
and install then skip to step 2.


In [24]:
$$async$$ = true;
var http = require('https');
var fs = require('fs');
var exec = require('child_process').exec;
var elevateLoc = process.cwd() + '\\selenium-act\\elevate.zip'
var expandedLoc = process.cwd() + '\\selenium-act\\elevate';
var elevateExecLoc = process.cwd() + '\\selenium-act\\elevate\\bin\\x86\\Release\\elevate.exe'
var dockerLoc = process.cwd() + '\\selenium-act\\InstallDocker.msi'
var logLoc = process.cwd() + '\\selenium-act\\log.txt'
console.log('checking if Docker is installed');
var docker = exec('docker ps', function (err, stdout, stderr) {
    if (stdout.indexOf('not found') > -1) {
        console.log('Docker not found, installing');
        if (process.platform === 'win32') {
            console.log('downloading Docker for Windows');
            var downloads = http.get('https://download.docker.com/win/stable/InstallDocker.msi', (r) => {
                r.pipe(fs.createWriteStream(dockerLoc)).on('finish', () => {
                    console.log('downloading jpassing/elevate');
                    http.get('https://github.com/jpassing/elevate/releases/download/1.0/elevate.zip', (r) => {
                        http.get(r.headers['location'], (r) => {
                            r.pipe(fs.createWriteStream(elevateLoc)).on('finish', () => {
                                console.log('downloaded docker.msi, elevate.zip');
                                console.log('extracting elevate.zip');
                                var expand = exec('powershell -c "Expand-Archive -Force ' + elevateLoc + ' ' + expandedLoc + '"', () => {
                                    console.log('running Docker install')
                                    var install = exec(elevateExecLoc + ' msiexec /i ' + dockerLoc + ' /qn /L*V! ' + logLoc, () => {
                                        $$done$$('installed Docker');
                                    });
                                    install.stdout.on('data', (d) => console.log(d));
                                    install.stderr.on('data', (d) => console.log(d));
                                });
                                expand.stdout.on('data', (d) => console.log(d));
                                expand.stderr.on('data', (d) => console.log(d));
                            });
                        });
                    });
                });
            });
        }
    } else {
        $$done$$('Docker is already installed');
    }
});
docker.stdout.on('data', (d) => console.log(d));
docker.stderr.on('data', (d) => console.log(d));


checking if Docker is installed
CONTAINER ID        IMAGE
               COMMAND                  CREATED             STATUS              PORTS                                                                                                                                            NAMES
e0fa18a623cb        act-selenium        "/opt/bin/entry_po..."   27 minutes ago      Up 27 minutes       0.0.0.0:3000->3000/tcp, 0.0.0.0:4200->4200/tcp, 0.0.0.0:4444->4444/tcp, 0.0.0.0:5900->5900/tcp, 0.0.0.0:6080->6080/tcp, 0.0.0.0:8888->8888/tcp   act-selenium



'Docker is already installed'

Create a Docker container with some dev tools:
* selenium (xvfb, x11vnc, novnc, mocha)
* source code (git, act.ecommerce, angular-cli)

Start from the selenium-chrome-debug image

In [25]:
$$async$$ = true;
var aptPart = `
FROM selenium/standalone-chrome-debug

RUN apt-get -qq update
RUN apt-get install -y --fix-missing git curl wget zip unzip vim dos2unix g++ python net-tools make 
RUN wget -O - https://deb.nodesource.com/setup_7.x | bash
RUN apt-get install -y nodejs
RUN nodejs -v

`;

var nodePart = `

ENV NODE_TLS_REJECT_UNAUTHORIZED 0
RUN npm install -g live-server babel-cli concurrently node-gyp nativescript@latest

COPY act.ecommerce /home/seluser/act.ecommerce
COPY mobile /home/seluser/mobile
ADD novnc /home/seluser/novnc

WORKDIR /home/seluser/act.ecommerce
RUN npm install
RUN npm run build

WORKDIR /home/seluser/mobile
RUN npm install

WORKDIR /home/seluser/
# modify the entry_point from selenium-docker to run our extra services
RUN dos2unix /home/seluser/novnc/utils/launch.sh
RUN sed -i '/wait \$NODE_PID/i/home/seluser/novnc/utils/launch.sh --vnc localhost:5900 &' /opt/bin/entry_point.sh
RUN sed -i '/wait \$NODE_PID/inpm --prefix /home/seluser/mobile run static &' /opt/bin/entry_point.sh

EXPOSE 4200
EXPOSE 4444
EXPOSE 3000

`;
var output = aptPart + nodePart;
var http = require('http');
// check if there is a local-npm server running on Brian's machine
http.get('http://SCO-100001269.ad.swiftpage.com:5080', (r) => {
    if (r.statusCode === 200) {
        console.log('local-npm found!');
        output = aptPart + `
RUN npm config set proxy http://sco-100001269.ad.swiftpage.com:8080
RUN npm config set proxy http://sco-100001269.ad.swiftpage.com:8080
RUN npm config set registry http://sco-100001269.ad.swiftpage.com:5080
RUN npm config set strict-ssl false
` + nodePart;
    }
    var fs = require('fs');
    fs.writeFile(process.cwd() + '\\selenium-act\\Dockerfile', output, function () {
        $$done$$('saved Dockerfile')
    });
});



local-npm found!


'saved Dockerfile'

## 2

Create a Docker file and project directory.

In [26]:
/* Your directory structure should look like this
file -What is Docker.ipynb
dir  -selenium-act
file |--Dockerfile
dir  |--act.ecommerce (from git)
dir  |--mobile       (git repo in progress)
dir  |--novnc        (https://github.com/novnc/noVNC/archive/master.zip)
*/

$$async$$ = true;
var exec = require('child_process').exec;
var rimraf = require('rimraf');
var novncDir = process.cwd() + '\\selenium-act\\novnc';
var actDir = process.cwd() + '\\selenium-act\\act.ecommerce';
rimraf(novncDir, function () {
    var cloneCmd = exec('git clone https://github.com/novnc/noVNC.git ' + novncDir, () => {
        rimraf(actDir, function () {
            var cloneActCmd = exec('git clone -b Buy_funnel https://github.com/Swiftpage/act.ecommerce.git ' + actDir, () => {
                $$done$$('cloned act.ecommerce, selenium-tests, and novnc');
            });
            cloneActCmd.stdout.on('data', (d) => console.log(d));
            cloneActCmd.stderr.on('data', (d) => console.log(d));
        });
    });
    cloneCmd.stdout.on('data', (d) => console.log(d));
    cloneCmd.stderr.on('data', (d) => console.log(d));
});


Cloning into 'C:\Users\brian.cullinan\Documents\jupytangular2\notebooks\selenium-act\novnc'...

Cloning into 'C:\Users\brian.cullinan\Documents\jupytangular2\notebooks\selenium-act\act.ecommerce'...



'cloned act.ecommerce, selenium-tests, and novnc'

## 3
Build the Docker image

In [27]:
$$async$$ = true;
var exec = require('child_process').exec;
var buildCmd = exec('docker build -t act-selenium ./selenium-act', () => {
    var imagesCmd = exec('docker images', () => {
        $$done$$('built Docker container')
    });
    imagesCmd.stdout.on('data', (d) => console.log(d));
    imagesCmd.stderr.on('data', (d) => console.log(d));
});
buildCmd.stdout.on('data', (d) => console.log(d));
buildCmd.stderr.on('data', (d) => console.log(d));





Step 1/26 : FROM selenium/standalone-chrome-debug

 ---> 870685c556e8
Step 2/26 : RUN apt-get -qq update

 ---> Using cache

 ---> aef2262dcd46

Step 3/26 : RUN apt-get install -y --fix-missing git curl wget zip unzip vim dos2unix g++ python net-tools make

 ---> Running in 77dec815dce3

Reading package lists...


Building dependency tree...

Reading state information...


unzip is already the newest version (6.0-20ubuntu1).
wget is already the newest version (1.17.1-1ubuntu1.2).
The following additional packages will be installed:

  binutils file g++-5 gcc gcc-5 git-man less libasan2 libatomic1 libc-dev-bin

  libc6-dev libcc1-0 libcilkrts5 libcurl3-gnutls liberror-perl libgcc-5-dev
  libgomp1 libgpm2 libitm1 liblsan0 libmagic1 libmpdec2 libmpx0 libpopt0
  libpython-stdlib libpython2.7-minimal libpython2.7-stdlib libpython3.5
  libpython3.5-minimal libpython3.5-stdlib libquadmath0 libstdc++-5-dev

  libtsan0 libubsan0 linux-libc-dev manpages manpages-dev mime-support
  openssh-clie

'built Docker container'

## 4

Run your Docker container

In [28]:
$$async$$ = true;
var exec = require('child_process').exec;
console.log('stopping current Docker');
var stopCmd = exec('docker stop act-selenium', () => {
    console.log('remove current Docker');
    var rmCmd = exec('docker rm act-selenium', () => {
        console.log('running new Docker');
        var runCmd = exec('docker run --shm-size=2g --name act-selenium -d -p 8888:8888 -p 6080:6080 -p 5900:5900 -p 4444:4444 -p 4200:4200 -p 3000:3000 act-selenium', () => {
            console.log('checking new Docker is running');
            var psCmd = exec('docker ps', () => {
                $$done$$('launched Docker container');
            });
            psCmd.stdout.on('data', (d) => console.log(d));
            psCmd.stderr.on('data', (d) => console.log(d));
        });
        runCmd.stdout.on('data', (d) => console.log(d));
        runCmd.stderr.on('data', (d) => console.log(d));
    });
    rmCmd.stdout.on('data', (d) => console.log(d));
    rmCmd.stderr.on('data', (d) => console.log(d));
});
stopCmd.stdout.on('data', (d) => console.log(d));
stopCmd.stderr.on('data', (d) => console.log(d));



stopping current Docker
act-selenium

remove current Docker
act-selenium

running new Docker
0ab11c326b827f44cdbb37e18dbaa039ca940216c6a6f5223635f0ef99b3b901

checking new Docker is running
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS                  PORTS                                                                                                                                            NAMES
0ab11c326b82        act-selenium        "/opt/bin/entry_po..."   1 second ago        Up Less than a second   0.0.0.0:3000->3000/tcp, 0.0.0.0:4200->4200/tcp, 0.0.0.0:4444->4444/tcp, 0.0.0.0:5900->5900/tcp, 0.0.0.0:6080->6080/tcp, 0.0.0.0:8888->8888/tcp   act-selenium



'launched Docker container'

## 5

Connect to VNC so we can see the test play out

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true) in a separate browser

In [29]:
// wait a few seconds for the container services to start
$$async$$ = true;
setTimeout(() => {
$$.html(
    '<div style="display:block; width:100%; padding-bottom:75%;position:relative;">' +
    '<iframe id="vnc"' +
        'style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;"' +
        'src="http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true"></iframe></div>');
}, 5000);



## 6

Run our first test by executing through docker

In [30]:
$$async$$ = true;
var exec = require('child_process').exec;
console.log('running selenium tests');
var runTest = exec('docker exec -t act-selenium npm --prefix /home/seluser/mobile run test', () => {
    $$done$$('test executed');
});
runTest.stdout.on('data', (d) => console.log(d));
runTest.stderr.on('data', (d) => console.log(d));



running selenium tests

> @ test /home/seluser/mobile
> wdio wdio.conf.js




ERROR: Couldn't load "wdio-mocha-framework" framework. You need to install it with `$ npm install wdio-mocha-framework`!
Error: Cannot find module 'wdio-mocha-framework'
    at Function.Module._resolveFilename (module.js:470:15)
    at Function.Module._load (module.js:418:25)
    at Module.require (module.js:498:17)
    at require (internal/module.js:20:19)
    at Runner.initialiseFramework (/home/seluser/mobile/node_modules/webdriverio/build/lib/runner.js:517:24)
    at Runner._callee$ (/home/seluser/mobile/node_modules/webdriverio/build/lib/runner.js:100:55)
    at tryCatch (/home/seluser/mobile/node_modules/regenerator-runtime/runtime.js:65:40)
    at Generator.invoke [as _invoke] (/home/seluser/mobile/node_modules/regenerator-runtime/runtime.js:303:22)
    at Generator.prototype.(anonymous function) [as next] (/home/seluser/mobile/node_modules/regenerator-runtime/runtime.js:117:21)
    at step (/home/selu

'test executed'